<a href="https://colab.research.google.com/github/andresnigenda/cpd_complaints_nlp/blob/krista/Naive_Bayes_EM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random as rnd
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from pprint import pprint

import regex as re
import random

In [0]:
text = pd.read_csv("training_with_text.csv")

In [0]:
text = text.loc[pd.notna(text.text_content)]

In [0]:
def clean_text(row):
  no_n = row['text_content'].replace('\n', ' ')
  no_special = re.sub("[^a-zA-Z0-9]+", ' ', no_n)
  return re.sub('\d', ' ', no_special)
text['text_content'] = text.apply(lambda x: clean_text(x), axis=1)

## Train test split and convert to batch

## Vectorizers and training

In [0]:
data = text['text_content']
labels = text['racial_slurs_xenophobic_remarks_']

In [0]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=3, max_df=0.9)
train_vec = vectorizer.fit_transform(data)
test_vec = vectorizer.transform(X_test)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_vec, np.array(labels), test_size=0.33, random_state=42)

In [84]:
clf = BernoulliNB(alpha=1e-8)
clf.fit(X_train, y_train)

BernoulliNB(alpha=1e-08, binarize=0.0, class_prior=None, fit_prior=True)

In [85]:
pred = clf.predict(test_vec)
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       115
           1       0.00      0.00      0.00         7

    accuracy                           0.94       122
   macro avg       0.47      0.50      0.49       122
weighted avg       0.89      0.94      0.91       122



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
feature_names = np.asarray(vectorizer.get_feature_names())
feature_names[np.argsort(clf.coef_[0])[-10:]]

array(['district', 'sustained', 'rank', 'allegation', 'violation', 'duty',
       'incident', 'used', 'action', 'location'], dtype='<U26')